In [7]:
import pandas as pd
import json
from web3 import Web3
from datetime import datetime, timedelta
import time
import sys
import json
from collections import Counter

# Map token names to contract addresses
token_address_map = {
    'rETH': '0xae78736Cd615f374D3085123A210448E74Fc6393'
    # fill with rest
}

# Map market name to "Pool" contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './AAVE_Pool_ABI.json')
    # fill with rest
}

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/7e4f5238262543919688a59d0fef4a1d'))

In [8]:
# Find block number closest to timestamp with binary search
def find_block_by_timestamp(target_timestamp):
    low, high = 0, w3.eth.block_number
    while low < high:
        mid = (low + high) // 2
        mid_block_timestamp = w3.eth.get_block(mid).timestamp
        if mid_block_timestamp < target_timestamp:
            low = mid+1
        else:
            high = mid
    return low


# Helper to convert block number to readable timestamp string
def get_block_timestamp(block_num):
    block = w3.eth.get_block(block_num)
    timestamp = block.timestamp
    
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')



# Lists transactions of supplying rETH to specified markets
def supply_transactions(markets = ['AAVE'], timeframe = 30, token = 'rETH'):
    """
    Returns all "Supply" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: supply_transactions(AAVE, 90, rETH) returns all supply transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique supply transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    # List for supply_lists of all markets
    all_market_supply_list = []
    
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Start processing supply API calls for each market
    for market in markets:
        # Make sure the requested market is supported
        if market in contract_address_abi_map.keys():
            # Set up contract address and ABI for each market
            contract_address, abi_filepath = contract_address_abi_map[market]
    
            # Web3 setup
            with open(abi_filepath) as f:
                abi = json.load(f)

            smart_contract = w3.eth.contract(address=contract_address, abi=abi)
            
            # Here we will need some sort of if/else or switching logic to process unique calls for each market
            # In the meantime, we simply check for the only supported market, 'AAVE'
            if market == 'AAVE':
                supplies = smart_contract.events.Supply().get_logs(fromBlock=st_block,
                                                           toBlock='latest',
                                                           argument_filters={'reserve': token_address})
                # Consider simplifying supply_list.. don't know what information future markets will return, 
                # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
                supply_list = [
                    {
                        'reserve': event['args']['reserve'],
                        'user': event['args']['user'],
                        'amount': event['args']['amount'],
                        'timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                        'log_index': event['logIndex'],
                        'transaction_index': event['transactionIndex'],
                        'transaction_hash': event['transactionHash'].hex(),
                        'block_hash': event['blockHash'].hex(),
                        'block_number': event['blockNumber']
                    }
                    for event in supplies
                ]
                # Append each markets supply list results to a single all markets list
                all_market_supply_list.append(supply_list)
        # The market wasn't in our dictionary of supported markets
        else:
            print("UNSUPPORTED MARKET ENTERED")
            
    # Push withdrawls into dataframe
    recent_supplies = pd.DataFrame(all_market_supply_list[0])
    #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
    return recent_supplies


"""
The above and below functions are extremely similar, the only difference being the API call they use, 
One function could have been used with some flow control and an additional parameter (User passes in "Borrow" or "Supply"
and the program runs accordingly); however, without knowing how future markets will behave, what their API calls will
exactly look like, and what they will return, it was decided to keep the two separate in case more complex logic is 
needed in handling either case specifically. 
"""



# Lists transactions of borrowing rETH to specified markets
def borrow_transactions(markets = ['AAVE'], timeframe = 30, token = 'rETH'):
    """
    Returns all "Borrow" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: borrow_transactions(AAVE, 90, rETH) returns all borrow transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique borrow transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    # List for borrow_lists of all markets
    all_market_borrow_list = []
    
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Start processing supply API calls for each market
    for market in markets:
        # Make sure the requested market is supported
        if market in contract_address_abi_map.keys():
            # Set up contract address and ABI for each market
            contract_address, abi_filepath = contract_address_abi_map[market]
    
            # Web3 setup
            with open(abi_filepath) as f:
                abi = json.load(f)

            smart_contract = w3.eth.contract(address=contract_address, abi=abi)
            
            # Here we will need some sort of if/else or switching logic to process unique calls for each market
            # In the meantime, we simply check for the only supported market, 'AAVE'
            if market == 'AAVE':
                borrows = smart_contract.events.Borrow().get_logs(fromBlock=st_block,
                                                           toBlock='latest',
                                                           argument_filters={'reserve': token_address})
                # Consider simplifying borrow_list.. don't know what information future markets will return, 
                # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
                borrow_list = [
                    {
                        'reserve': event['args']['reserve'],
                        'user': event['args']['user'],
                        'amount': event['args']['amount'],
                        'timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                        'log_index': event['logIndex'],
                        'transaction_index': event['transactionIndex'],
                        'transaction_hash': event['transactionHash'].hex(),
                        'block_hash': event['blockHash'].hex(),
                        'block_number': event['blockNumber']
                    }
                    for event in borrows
                ]
                # Append each markets supply list results to a single all markets list
                all_market_borrow_list.append(borrow_list)
        # The market wasn't in our dictionary of supported markets
        else:
            print("UNSUPPORTED MARKET ENTERED")
            
    # Push withdrawls into dataframe
    recent_borrows = pd.DataFrame(all_market_borrow_list[0])
    #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
    return recent_borrows
    
                

            
            
            
            
    
    
    
    

In [9]:
recent_supply = supply_transactions()
recent_supply

,reserve,user,amount,timestamp,log_index,transaction_index,transaction_hash,block_hash,block_number
0,0xae78736Cd615f374D3085123A210448E74Fc6393,0x1352Bf00ACB9876088e1C5c912Ffd43497423479,3500000000000000000,2024-03-05 12:37:59,254,85,0x071445709cfa9343565894463dd9926e899850fcd512...,0x033d7b5640f40e18c16fff15ebb6a6783a78b7c706e0...,19369013
1,0xae78736Cd615f374D3085123A210448E74Fc6393,0xE0a97c2634C0b1D2530B2F92E46697460692cA0c,7000000000000000000,2024-03-06 15:01:11,334,180,0x4c740f9cf7db453995d5edd2e13bec6f4600c45aa027...,0x6e0c5bd6f64acd78ec603278c8d0d2998b45f5310e64...,19376892
2,0xae78736Cd615f374D3085123A210448E74Fc6393,0x25156aD7Dddf95b8A8C68B289709e65821974c68,7261638663453844432,2024-03-07 13:05:23,261,91,0x96861ec4f315d1693399d5f4d4bba9b3b869445dac86...,0x6087019daf99d4c401297ed407ff0a9b566139c7a5c0...,19383478
3,0xae78736Cd615f374D3085123A210448E74Fc6393,0x25156aD7Dddf95b8A8C68B289709e65821974c68,6930160000000000000,2024-03-07 13:17:35,403,226,0x2f70f7a5d273c2c988f209e5cb9892d677d2321846ea...,0x3272d209275eccad2d010252e1d97146fab15bedffe1...,19383539
4,0xae78736Cd615f374D3085123A210448E74Fc6393,0xE0a97c2634C0b1D2530B2F92E46697460692cA0c,8000000000000000000,2024-03-08 03:56:11,146,105,0x40eaf9c2609da34b348b66b2b2578260147768fe0451...,0x5b80faf85a9e46c4f25da6c847a16f8243dcc4c543b7...,19387905
...,...,...,...,...,...,...,...,...,...
104,0xae78736Cd615f374D3085123A210448E74Fc6393,0xcCdAE12162566E3f29fEfA7Bf7F5b24C644493b5,4420726939893561797,2024-04-03 18:46:23,214,49,0xeed2cc40c94e1b863c90d281f1f67241371c00995a6a...,0x8916d4d5f3e30bda7e7541018a1bdbaf0a60a7952916...,19577123
105,0xae78736Cd615f374D3085123A210448E74Fc6393,0xcCdAE12162566E3f29fEfA7Bf7F5b24C644493b5,4067958003907258649,2024-04-03 18:47:23,182,70,0xe3766b892246d850ac6ff1e3b055f13662c1f12a023b...,0x20b51c107ba347f7be90c6f38a6a8e8e859aa0b1dfaa...,19577128
106,0xae78736Cd615f374D3085123A210448E74Fc6393,0xcCdAE12162566E3f29fEfA7Bf7F5b24C644493b5,3743266269403108172,2024-04-03 18:54:59,173,77,0x622d73b38185a2a324e9b64a13a535116b354958428a...,0xadbccb5940bceed96fa7a864dc37c1a9df5efa70be8f...,19577164
107,0xae78736Cd615f374D3085123A210448E74Fc6393,0xcCdAE12162566E3f29fEfA7Bf7F5b24C644493b5,348397173289189013,2024-04-03 19:00:35,343,107,0x9cc74fca284593e26250830ec78b8ec27dfba8250a73...,0x9b02256262f3d0e412db8b9ef7e70d1c6910929415f3...,19577192
